In [6]:
# have a pytorch mesh
from pytorch3d.structures import Meshes
import torch

def get_signed_tet_volume(triangle):
    """
    triangle is a Tensor of 3 x 3, 3 vertices each with 3 coordinates
    """
    v1,v2,v3 = triangle
    return torch.dot(v1,torch.cross(v2,v3))/6.0 # could also move this later

def get_volume(mesh: Meshes):
    volume = 0
    for triangle in mesh.faces_list():
        volume += get_signed_tet_volume(triangle)
    return volume   

def get_parallelepiped_volume(triangle):
    v1,v2,v3 = triangle
    return torch.dot(v1,torch.cross(v2,v3)) # could also move this later

def get_volume(mesh: Meshes):
    volume = 0.0
    for triangle in mesh.faces_list():
        volume += get_parallelepiped_volume(triangle)
    return volume / 6.0


In [7]:

def get_signed_tet_volume(face_vertices: torch.Tensor) -> torch.Tensor:
    """
    Compute signed tetrahedron volumes for a batch of faces.

    Args:
        face_vertices (torch.Tensor): Tensor of shape (F, 3, 3), where
                                      F is the number of faces, and each face
                                      consists of 3 vertices in 3D.

    Returns:
        torch.Tensor: A tensor of shape (F,) containing signed volumes.
    """
    v0, v1, v2 = face_vertices[:, 0, :], face_vertices[:, 1, :], face_vertices[:, 2, :]
    
    # Compute determinant of the 3x3 matrix [v0, v1, v2]
    volumes = torch.det(torch.stack([v0, v1, v2], dim=-1)) / 6.0  # Shape: (F,)

    return volumes

def get_volume_batch(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    mesh_to_face = meshes.mesh_to_faces_packed_first_idx()  # Index of first face per mesh
    n_meshes = len(meshes)
    volumes = torch.zeros(n_meshes, device=verts_packed.device)

    for i in range(n_meshes):
        start = mesh_to_face[i]
        end = start + meshes.num_faces_per_mesh()[i]
        face_vertices = verts_packed[faces_packed[start:end]]  # (F, 3, 3)
        volumes[i] = get_signed_tet_volume(face_vertices).sum()  # Sum over all faces

    return volumes  # Returns a tensor of shape (num_meshes,)


def get_volume_scatter_add(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    
    face_vertices = verts_packed[faces_packed]  # (sum(F_i), 3, 3)
    face_volumes = get_signed_tet_volume(face_vertices)  # (sum(F_i),)
    n_meshes = len(meshes)

    # Sum per mesh using torch_scatter
    volumes = torch.zeros(n_meshes, device=verts_packed.device, dtype=face_volumes.dtype)
    volumes.scatter_add_(0, meshes.faces_packed_to_mesh_idx(), face_volumes)
    return volumes  # Shape: (num_meshes,)

In [8]:
def signed_volume_gradient(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    p0 = verts_packed[faces_packed[:, 0]]  # (sum(F_i), 3)
    p1 = verts_packed[faces_packed[:, 1]]
    p2 = verts_packed[faces_packed[:, 2]]

    # Compute cross products for gradients
    grad_p0 = torch.cross(p1, p2, dim=1) / 6.0
    grad_p1 = torch.cross(p2, p0, dim=1) / 6.0
    grad_p2 = torch.cross(p0, p1, dim=1) / 6.0

    # Scatter gradients back to vertices
    grad_verts = torch.zeros_like(verts_packed)
    grad_verts.scatter_add_(0, faces_packed[:, 0].unsqueeze(1).expand(-1, 3), grad_p0)
    grad_verts.scatter_add_(0, faces_packed[:, 1].unsqueeze(1).expand(-1, 3), grad_p1)
    grad_verts.scatter_add_(0, faces_packed[:, 2].unsqueeze(1).expand(-1, 3), grad_p2)

    return grad_verts

In [9]:
vertices = torch.tensor([[0.0, 0.0, 1.0],  # Vertex 0
                         [1.0, 0.0, 0.0],  # Vertex 1
                         [0.0, 1.0, 0.0],  # Vertex 2
                         [1.0, 1.0, 0.0]]) # Vertex 3

# Define two faces (two triangles)
faces = torch.tensor([[0, 1, 2],  # Face 1
                      [1, 2, 3]]) # Face 2

# Create the Meshes object
mesh = Meshes(verts=[vertices], faces=[faces])
gradients = signed_volume_gradient(mesh)

# Output the gradients for each vertex
print("Signed volume gradients:\n", gradients)


Signed volume gradients:
 tensor([[ 0.0000,  0.0000,  0.1667],
        [ 0.1667,  0.0000, -0.1667],
        [ 0.0000,  0.1667, -0.1667],
        [ 0.0000,  0.0000,  0.1667]])


In [23]:
verts = torch.tensor([
    [-0.5, -0.5, -0.5],  # 0
    [ 0.5, -0.5, -0.5],  # 1
    [ 0.5,  0.5, -0.5],  # 2
    [-0.5,  0.5, -0.5],  # 3
    [-0.5, -0.5,  0.5],  # 4
    [ 0.5, -0.5,  0.5],  # 5
    [ 0.5,  0.5,  0.5],  # 6
    [-0.5,  0.5,  0.5]   # 7
], dtype=torch.float32)
faces = torch.tensor([
    [0, 2, 1], [0, 3, 2],  # Front
    [1, 6, 5], [1, 2, 6],  # Right
    [5, 7, 4], [5, 6, 7],  # Back
    [4, 3, 0], [4, 7, 3],  # Left
    [3, 6, 2], [3, 7, 6],  # Top
    [4, 1, 5], [4, 0, 1]   # Bottom
], dtype=torch.int64)

verts2 = torch.tensor([
    [0,0,1],
    [1,0,0],
    [0,1,0], 
    [0,0,0]], dtype=torch.float32)
faces2 = torch.tensor([
    [3, 1, 2],
    [3, 1, 0],
    [3, 0, 2],
    [0, 1, 2]
], dtype=torch.int64)

                      
# Create batched cube mesh (batch size 1)
mesh = Meshes(verts=[verts,verts2], faces=[faces,faces2])
mesh.verts_normals_packed()


tensor([[-0.4082, -0.4082, -0.8165],
        [ 0.6667, -0.6667, -0.3333],
        [ 0.4082,  0.4082, -0.8165],
        [-0.6667,  0.6667, -0.3333],
        [-0.6667, -0.6667,  0.3333],
        [ 0.4082, -0.4082,  0.8165],
        [ 0.6667,  0.6667,  0.3333],
        [-0.4082,  0.4082,  0.8165],
        [ 0.0000,  0.0000,  1.0000],
        [ 0.4472,  0.0000,  0.8944],
        [ 0.0000,  0.4472,  0.8944],
        [-0.5774, -0.5774,  0.5774]])

In [24]:
volume = get_volume_scatter_add(mesh)
print("Volume:", volume)

Volume: tensor([1.0000, 0.1667])


In [25]:
for m in mesh:
    print(m.verts_packed())

tensor([[-0.5000, -0.5000, -0.5000],
        [ 0.5000, -0.5000, -0.5000],
        [ 0.5000,  0.5000, -0.5000],
        [-0.5000,  0.5000, -0.5000],
        [-0.5000, -0.5000,  0.5000],
        [ 0.5000, -0.5000,  0.5000],
        [ 0.5000,  0.5000,  0.5000],
        [-0.5000,  0.5000,  0.5000]])
tensor([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]])


In [12]:
get_signed_tet_volume(torch.tensor([[[0,0,1],[1,0,0],[0,1,0]]],dtype=float))

tensor([0.1667], dtype=torch.float64)